In [3]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm
import shutil
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('augmentation.log'),
        logging.StreamHandler()
    ]
)

def create_augmentation_pipeline():
    """Создаем пайплайн аугментаций"""
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1),
            A.GaussNoise(p=1),
        ], p=0.5),
        A.Resize(224, 224)
    ])

def augment_dataset(train_dir, num_augmentations=3):
    """Аугментация изображений в директории train"""
    # Получаем список всех изображений в train
    all_image_files = [f for f in os.listdir(os.path.join(train_dir, 'images')) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # Фильтруем только оригинальные изображения (без '_aug_' в имени)
    original_image_files = [f for f in all_image_files if '_aug_' not in f]
    
    logging.info(f"Found {len(original_image_files)} original images in train directory")
    
    # Создаем аугментатор
    transform = create_augmentation_pipeline()
    
    # Генерируем аугментации для каждого оригинального изображения
    logging.info(f"Generating {num_augmentations} augmentations per image...")
    for img_file in tqdm(original_image_files):
        # Загружаем изображение
        img_path = os.path.join(train_dir, 'images', img_file)
        image = cv2.imread(img_path)
        
        if image is None:
            logging.error(f"Could not load {img_file}")
            continue
            
        # Генерируем аугментации
        for i in range(num_augmentations):
            # Применяем аугментации
            transformed = transform(image=image)
            
            # Сохраняем аугментированное изображение
            aug_img_file = f"{img_file.rsplit('.', 1)[0]}_aug_{i}.{img_file.rsplit('.', 1)[1]}"
            cv2.imwrite(
                os.path.join(train_dir, 'images', aug_img_file),
                transformed['image']
            )

    # Проверяем результат
    result_files = [f for f in os.listdir(os.path.join(train_dir, 'images')) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    logging.info(f"\nAugmentation completed!")
    logging.info(f"Original images in train: {len(original_image_files)}")
    logging.info(f"Generated augmentations: {len(result_files) - len(original_image_files)}")
    logging.info(f"Total images after augmentation: {len(result_files)}")

if __name__ == "__main__":
    train_dir = "train"  # Теперь работаем только с train директорией
    augment_dataset(train_dir, num_augmentations=3)

2025-01-21 12:14:22,384 - INFO - Found 100 original images in train directory
2025-01-21 12:14:22,387 - INFO - Generating 3 augmentations per image...
100%|██████████| 100/100 [00:03<00:00, 29.59it/s]
2025-01-21 12:14:25,771 - INFO - 
Augmentation completed!
2025-01-21 12:14:25,772 - INFO - Original images in train: 100
2025-01-21 12:14:25,772 - INFO - Generated augmentations: 300
2025-01-21 12:14:25,773 - INFO - Total images after augmentation: 400
